# Setup

In [ ]:
# NN and related metrics
from tensorflow import keras
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# General
import numpy as np
import os
import random
import pandas as pd

# Image and visualiztaion
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from PIL import Image

In [ ]:
# Specifying relevant paths
WORKING_DIRECTORY = os.getcwd()
DEPENDENCIES = os.path.join(WORKING_DIRECTORY, 'dependencies')
MODELS = os.path.join(DEPENDENCIES, 'models')
TEST = os.path.join(DEPENDENCIES, 'cnn_data', 'test')
YOLO = os.path.join(DEPENDENCIES, 'yolov5')
YOLO_MODEL = os.path.join(YOLO, 'exam', 'models')

## Importing CNN Models

In [ ]:
Xception = keras.models.load_model(f'{MODELS}/Xception_model_v2.keras')
TinyVGG = keras.models.load_model(f'{MODELS}/TinyVGG_Model_v2.keras')

In [ ]:
TinyVGG.get_compile_config()

## Importing Test Data

In [ ]:
img_height, img_width = 250, 250

# Importing as tensorflow dataset for easy evaluation
tensorflow_test = tf.keras.utils.image_dataset_from_directory(
    TEST,
    labels='inferred',
    color_mode='rgb',
    seed=42,
    batch_size=32,
    image_size=(img_height, img_width),)

# class names
class_names = tensorflow_test.class_names
class_index = {index: name for index, name in enumerate(class_names)}
print(f'Class index of class names {class_index}')

In [ ]:
### To make image visualization easier, and for making predictions on test set, we will import them also as arrays

# We need to reshape images, so that our model will accept it (It expects batch-inputs)
new_shape = (1, 250, 250, 3)

# Extracting image path using OS
bicycle_array = [os.path.join(TEST, 'bicycle', img) for img in os.listdir(os.path.join(TEST, 'bicycle'))]
# Reshaping picture so that models will accept them (1,250,250, 3), assigning label and adding PIL.Image render to tuple, so x, y, img = bicycle[i] 
bicycle = [(np.reshape(cv2.imread(img), new_shape), 0, Image.open(img)) for img in bicycle_array]


##Same for the rest...
car_array = [os.path.join(TEST, 'car', img) for img in os.listdir(os.path.join(TEST, 'car'))]
car = [(np.reshape(cv2.imread(img), new_shape), 1, Image.open(img)) for img in car_array]

person_array = [os.path.join(TEST, 'person', img) for img in os.listdir(os.path.join(TEST, 'person'))]
person = [(np.reshape(cv2.imread(img), new_shape), 2, Image.open(img)) for img in person_array]

# Combining

imgs = bicycle_array + car_array + person_array
test_set = bicycle + car + person

random.shuffle(test_set)

# Tiny VGG

## Tiny VGG Summary

In [ ]:
TinyVGG.summary()

In [ ]:
tf.keras.utils.plot_model(TinyVGG)

## Evaluate on test set

In [ ]:
TinyVGG_results = TinyVGG.evaluate(tensorflow_test)

print(f'Sparse Categorical Cross Entropy: {round(TinyVGG_results[0], 2)}\nPercentage of correct predictions: {TinyVGG_results[1]:0.1%}')

### F1, Precision, Recall, Confusion Matrix

In [ ]:
# Extracting labels and predictions for every image
vgg_y_true = []
vgg_y_pred = []

for img in test_set:
    x, y, i = img
    
    pred = TinyVGG(x)
    
    vgg_y_true.append(y)
    vgg_y_pred.append(np.argmax(pred, axis=1)[0])
    
# Transforming to arrays to pass through sklearn.metrics
vgg_y_true = np.array(vgg_y_true)
vgg_y_pred = np.array(vgg_y_pred)

In [ ]:
# Calculate metrics
vgg_prf = precision_recall_fscore_support(vgg_y_true, vgg_y_pred)

# Transform into table
table =  [['precision'] + list(vgg_prf[0]),
['recall'] + list(vgg_prf[1]),
['fscore'] + list(vgg_prf[2])]

In [ ]:
# Print in to tabulate form
print(tabulate(table, headers=[
      'Metric', class_index[0], class_index[1], class_index[2]]))


In [ ]:
# Calculating confusion matrix
fig, ax = plt.subplots(figsize=(12,12))
vgg_cm = confusion_matrix(vgg_y_true, vgg_y_pred, normalize='pred')

sns.heatmap(vgg_cm, cmap='Blues', ax=ax, annot=True)

# Set the tick labels for x-axis
x_ticks = ax.get_xticks().tolist()

ax.set_xticklabels([class_index[int(x)] if int(x) in class_index else x for x in x_ticks])

# Set the tick labels for y-axis
y_ticks = ax.get_yticks().tolist()
ax.set_yticklabels([class_index[int(y)] if int(y) in class_index else y for y in y_ticks])


plt.show()

## Predictions

In [ ]:
# Visualizing Predictions Across Images
fig, axs = plt.subplots(3, 2, figsize=(15, 15))

# Looping through ax's
for ax in axs.flatten():

    # Get random image position
    index = random.randint(1, len(test_set)-1)

    # Extract images and labels from test set
    x, y_true, i = test_set[index]

    # Generate Prediction
    y_pred = TinyVGG(x)

    # Get class with highest probability
    y_pred_arg = np.argmax(y_pred, axis=1)[0]

    y_pred_prob = np.array(y_pred)[0][y_pred_arg]

    y_pred_prob = round(float(y_pred_prob), 2)

    ax.imshow(i)
    ax.set_title(
        f'Predicted label: {class_index[y_pred_arg]} ({y_pred_prob}) True label: {class_index[y_true]}')
    ax.axis('off')

plt.tight_layout()
plt.show()


# Xception

## Xception Summary

In [ ]:
Xception.summary()

In [ ]:
tf.keras.utils.plot_model(Xception)

## Evaluate on test set

In [ ]:
Xception_results = Xception.evaluate(tensorflow_test)

print(
    f'Sparse Categorical Cross Entropy Loss: {Xception_results[0]:0.2}\nPercentage of correct predictions: {Xception_results[1]:0.1%}')


### F1, Precision, Recall, Confusion Matrix

In [ ]:
# Extracting labels and predictions for every image
xc_y_true = []
xc_y_pred = []

for img in test_set:
    x, y, i = img
    
    pred = Xception(x)
    
    xc_y_true.append(y)
    xc_y_pred.append(np.argmax(pred, axis=1)[0])
    
# Transforming to arrays to pass through sklearn.metrics
xc_y_true = np.array(xc_y_true)
xc_y_pred = np.array(xc_y_pred)

In [ ]:
Xception_prf = precision_recall_fscore_support(xc_y_true, xc_y_pred)

table =  [['precision'] + list(Xception_prf[0]),
['recall'] + list(Xception_prf[1]),
['fscore'] + list(Xception_prf[2])]

In [ ]:
print(tabulate(table, headers=['Metric', class_index[0], class_index[1], class_index[2]]))

In [ ]:
# Calculating confusion matrix
fig, ax = plt.subplots(figsize=(12,12))
cm = confusion_matrix(xc_y_true, xc_y_pred, normalize='pred')

sns.heatmap(cm, cmap='Blues', ax=ax, annot=True)

# Set the tick labels for x-axis
x_ticks = ax.get_xticks().tolist()

ax.set_xticklabels([class_index[int(x)] if int(x) in class_index else x for x in x_ticks])

# Set the tick labels for y-axis
y_ticks = ax.get_yticks().tolist()
ax.set_yticklabels([class_index[int(y)] if int(y) in class_index else y for y in y_ticks])


plt.show()

## Making Predictions 

In [ ]:
# Visualizing Predictions Across Images
fig, axs = plt.subplots(3,2, figsize=(15,15))

# Looping through ax's
for ax in axs.flatten():
    
    # Get random image position
    index = random.randint(1, len(test_set)-1)
    
    # Extract images and labels from test set
    x, y_true, i = test_set[index]
    
    # Generate Prediction
    y_pred = Xception(x)
    
    # Get class with highest probability
    y_pred_arg = np.argmax(y_pred, axis=1)[0]
    
    y_pred_prob = np.array(y_pred)[0][y_pred_arg]
    
    y_pred_prob = round(float(y_pred_prob), 2)
    
    ax.imshow(i)
    ax.set_title(f'Predicted label: {class_index[y_pred_arg]} ({y_pred_prob}) True label: {class_index[y_true]}')
    ax.axis('off')

plt.tight_layout()
plt.show()